# Cassandra datasource example with Spark 1.6
The following notebook shows how to use the Cassandra datasource connector from Scala.

In [1]:
import com.datastax.spark.connector._
import com.datastax.spark.connector.cql._
import org.apache.spark.{SparkConf, SparkContext}
import org.apache.spark.sql.SQLContext

#### We need to stop the default SparkContext (sc) and create one configured for Cassandra.

In [2]:
sc.stop

#### Configure Spark to use the Cassandra connector

In [3]:
val conf = (new SparkConf(true)
            .setMaster("local[*]")
            .setAppName("Jupyter Scala")
            .set("spark.cassandra.connection.host", "cassandra")
            .set("spark.cassandra.input.consistency.level", "LOCAL_ONE")
            )
val sc = new SparkContext(conf)
val sqlContext = new SQLContext(sc)

#### Create the test keyspace and table and insert a test row.

In [4]:
CassandraConnector(conf).withSessionDo { session =>
                                        session.execute("CREATE KEYSPACE IF NOT EXISTS test WITH replication={'class':'SimpleStrategy', 'replication_factor':1}")
                                        session.execute("CREATE TABLE IF NOT EXISTS test.users (username text PRIMARY KEY, emails SET<text>)")
                                        session.execute("INSERT INTO test.users (username,emails) VALUES('someone',{'someone@email.com', 's@email.com'})")
                                       }

ResultSet[ exhausted: true, Columns[]]

#### Load the test data from Cassandra.

In [5]:
val df = sqlContext.read.format("org.apache.spark.sql.cassandra").options(Map("cluster" -> "Cluster Dock", "table" -> "users", "keyspace" -> "test")).load()

In [6]:
df.show()

+--------+--------------------+
|username|              emails|
+--------+--------------------+
| someone|[s@email.com, som...|
+--------+--------------------+

